# Data Wrangling Challenge
### Pull and manipulate the API data

The point of this exercise is to try data enrichment with data from external APIs. We are going to take data about car crashes in Monroe County, Indiana from 2003 to 2015 and try to figure out the weather during the accident and how many bars there are in the area. We will work with two different APIs during this challenge:

- Foursquare API
- World Weather Online API

We will try to find correlations between the severity of crash and weather/number of bars in the area. To indicate the severity of a crash, we will use column `Injury Type`.

In [49]:
import pandas as pd
import requests as re

## Data

The data for this exercise can be found [here](https://drive.google.com/file/d/1_KF9oIJV8cB8i3ngA4JPOLWIE_ETE6CJ/view?usp=sharing).

Just run the cells below to get your data ready. Little help from us.


In [109]:
# had to specify the correct coding
crash_data = pd.read_csv('data/monroe_county_crash_data.csv', encoding='ISO-8859-1')
crash_data.tail(3)

,Master Record Number,Year,Month,Day,Weekend?,Hour,Collision Type,Injury Type,Primary Factor,Reported_Location,Latitude,Longitude
10396,902003089,2013,4,5,Weekday,1200,2-Car,No injury/unknown,FOLLOWING TOO CLOSELY,S CURRY & SR 48 RD,39.16466,-86.58292
10397,902068276,2013,8,5,Weekday,1700,2-Car,No injury/unknown,FOLLOWING TOO CLOSELY,CURRY & W 3RD ST,39.16466,-86.58292
10398,902084557,2013,9,3,Weekday,700,2-Car,Non-incapacitating,FAILURE TO YIELD RIGHT OF WAY,CURRY PIKE & W THIRD,39.16466,-86.58292


# Foursquare API

Foursquare API documentation is [here](https://developer.foursquare.com/)

1. Start a foursquare application and get your keys.
2. For each crash, create the function **get_venues** that will pull bars in the radius of 5km around the crash

#### example
`get_venues('48.146394, 17.107969')`

3. Find a relationship (if there is any) between number of bars in the area and severity of the crash.

HINTs: 
- check out python package "foursquare" (no need to send HTTP requests directly with library `requests`)
- **categoryId** for bars and nightlife needs to be found in the [foursquare API documentation](https://developer.foursquare.com/docs/api-reference/venues/search/)

In [110]:
for i in range(0, crash_data.shape[0]):
    longitude, latitude = crash_data[["Latitude", "Longitude"]].iloc[i]

longitude = 23.4058
longitude = f"{longitude:.2f}"

print(longitude)
type(longitude)

23.41


str

In [111]:
# function to return a response from the query (venue, in this case) and the point of reference (long/lang) with radius in m
def get_venues(venue, longitude, latitude, radius):

    url = "https://api.foursquare.com/v3/places/search"

    longitude = f"{longitude:.2f}"
    latitude = f"{latitude:.2f}"

    params = {
        "query" : venue,
        "ll" : longitude + "," + latitude,
        "sort" : "DISTANCE",
        "radius" : str(radius)
    }

    headers = {
        "Accept" : "application/json",
        "Authorization" : "fsq3P3rZR2Kb2ccaHpbuTMdwWCtYF3fIp1frLeqT2saiS0M="
    }

    response = re.get(url, params=params, headers=headers)

    return response

In [55]:
bar_data_example = get_venues("bar", 48.146394, 17.107969, 5000).json()

In [67]:
bar_data_example['results']

[{'fsq_id': '5baf3efbc66666002c063861',
  'categories': [{'id': 13012,
    'name': 'Hookah Bar',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/hookahbar_',
     'suffix': '.png'}}],
  'chains': [],
  'distance': 78,
  'geocodes': {'main': {'latitude': 48.149291, 'longitude': 17.110026}},
  'link': '/v3/places/5baf3efbc66666002c063861',
  'location': {'address': 'Námestie 1. mája 4',
   'country': 'SK',
   'cross_street': '',
   'formatted_address': 'Námestie 1. mája 4, 811 06 Bratislava',
   'locality': 'Bratislava',
   'postcode': '811 06',
   'region': 'Bratislava Region'},
  'name': 'Vice City Shisha Bar & Lounge',
  'related_places': {},
  'timezone': 'Europe/Bratislava'},
 {'fsq_id': '53c2d3e2498eda35d7854d64',
  'categories': [{'id': 13003,
    'name': 'Bar',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/pub_',
     'suffix': '.png'}}],
  'chains': [],
  'distance': 85,
  'geocodes': {'main': {'latitude': 48.149235, 'longitude

In [84]:
pd.json_normalize(
    data=bar_data_example,
    record_path="results",
)

,fsq_id,categories,chains,distance,link,name,timezone,geocodes.main.latitude,geocodes.main.longitude,location.address,location.country,location.cross_street,location.formatted_address,location.locality,location.postcode,location.region
0,5baf3efbc66666002c063861,"[{'id': 13012, 'name': 'Hookah Bar', 'icon': {...",[],78,/v3/places/5baf3efbc66666002c063861,Vice City Shisha Bar & Lounge,Europe/Bratislava,48.149291,17.110026,Námestie 1. mája 4,SK,,"Námestie 1. mája 4, 811 06 Bratislava",Bratislava,811 06,Bratislava Region
1,53c2d3e2498eda35d7854d64,"[{'id': 13003, 'name': 'Bar', 'icon': {'prefix...",[],85,/v3/places/53c2d3e2498eda35d7854d64,Smile Bar & Caffe,Europe/Bratislava,48.149235,17.109996,NaN,SK,,,NaN,NaN,NaN
2,610530bd07179e0960d482b2,"[{'id': 13006, 'name': 'Beer Bar', 'icon': {'p...",[],125,/v3/places/610530bd07179e0960d482b2,Mešuge Craft Beer Bar,Europe/Bratislava,48.148932,17.110533,Vysoká 15,SK,,"Vysoká 15, 811 06 Bratislava",Bratislava,811 06,Bratislava Region
3,57443442498e13105b9db843,"[{'id': 13057, 'name': 'Gastropub', 'icon': {'...",[],133,/v3/places/57443442498e13105b9db843,Skupinová Terapia,Europe/Bratislava,48.149856,17.111783,Kollárovo nám.,SK,,"Kollárovo nám., Bratislava",Bratislava,NaN,Bratislava Region
4,4f4d5ed7e4b0272872a3782a,"[{'id': 13016, 'name': 'Lounge', 'icon': {'pre...",[],166,/v3/places/4f4d5ed7e4b0272872a3782a,EVENT bar & restaurant,Europe/Bratislava,48.151471,17.110419,"Jozefská 2982/7, 81106 Bratislava - Staré Mest...",SK,,"Jozefská 2982/7, 81106 Bratislava - Staré Mest...",Bratislava,811 06,Bratislava Region
5,4e33c4cdb0fb17f64f8695db,"[{'id': 13002, 'name': 'Bakery', 'icon': {'pre...",[],177,/v3/places/4e33c4cdb0fb17f64f8695db,Minute - Fresh Food Bar,Europe/Bratislava,48.149603,17.112317,Kollarove namestie,SK,,"Kollarove namestie, Bratislava",Bratislava,NaN,Bratislava Region
6,61f56cf887001436dbebcd93,"[{'id': 13009, 'name': 'Cocktail Bar', 'icon':...",[],238,/v3/places/61f56cf887001436dbebcd93,MYST BAR,Europe/Bratislava,48.149131,17.112938,15 Kollárovo námestie,SK,,"15 Kollárovo námestie, 811 06 Bratislava",Bratislava,811 06,Bratislava Region
7,4b446645f964a5208bf425e3,"[{'id': 13341, 'name': 'Slovak Restaurant', 'i...",[],247,/v3/places/4b446645f964a5208bf425e3,1. Slovak pub,Europe/Bratislava,48.148398,17.112310,Obchodná 62,SK,NaN,"Obchodná 62, 811 02 Bratislava",Bratislava,811 02,Bratislava Region
8,5233620d11d2e9c6c180d1ac,"[{'id': 13034, 'name': 'Café', 'icon': {'prefi...",[],247,/v3/places/5233620d11d2e9c6c180d1ac,Bar BaRon,Europe/Bratislava,48.147839,17.110789,NaN,SK,,,NaN,NaN,NaN
9,5a8dc72af4b52557932d20f4,"[{'id': 13006, 'name': 'Beer Bar', 'icon': {'p...",[],256,/v3/places/5a8dc72af4b52557932d20f4,Kollarko,Europe/Bratislava,48.149161,17.113223,Kollárovo námestie 23,SK,,"Kollárovo námestie 23, 811 06 Bratislava",Bratislava,811 06,Bratislava Region


In [103]:
bar_data = pd.json_normalize(
    data=bar_data_example["results"],
    record_path="categories",
    meta=["distance", "name", ["geocodes", "main", "latitude"], ["geocodes", "main", "longitude"]],
    record_prefix="cat."
    ).drop(columns = ["cat.id", "cat.icon.prefix", "cat.icon.suffix"])

In [106]:
bar_data.rename(columns={"cat.name" : "cat_id", "geocodes.main.latitude" : "latitude", "geocodes.main.longitude" : "longitude"})

,cat_id,distance,name,latitude,longitude
0,Hookah Bar,78,Vice City Shisha Bar & Lounge,48.149291,17.110026
1,Bar,85,Smile Bar & Caffe,48.149235,17.109996
2,Beer Bar,125,Mešuge Craft Beer Bar,48.148932,17.110533
3,Gastropub,133,Skupinová Terapia,48.149856,17.111783
4,Lounge,166,EVENT bar & restaurant,48.151471,17.110419
5,Bakery,177,Minute - Fresh Food Bar,48.149603,17.112317
6,Cocktail Bar,238,MYST BAR,48.149131,17.112938
7,Slovak Restaurant,247,1. Slovak pub,48.148398,17.11231
8,Café,247,Bar BaRon,48.147839,17.110789
9,Beer Bar,256,Kollarko,48.149161,17.113223


In [108]:
bar_data

,cat.name,distance,name,geocodes.main.latitude,geocodes.main.longitude
0,Hookah Bar,78,Vice City Shisha Bar & Lounge,48.149291,17.110026
1,Bar,85,Smile Bar & Caffe,48.149235,17.109996
2,Beer Bar,125,Mešuge Craft Beer Bar,48.148932,17.110533
3,Gastropub,133,Skupinová Terapia,48.149856,17.111783
4,Lounge,166,EVENT bar & restaurant,48.151471,17.110419
5,Bakery,177,Minute - Fresh Food Bar,48.149603,17.112317
6,Cocktail Bar,238,MYST BAR,48.149131,17.112938
7,Slovak Restaurant,247,1. Slovak pub,48.148398,17.11231
8,Café,247,Bar BaRon,48.147839,17.110789
9,Beer Bar,256,Kollarko,48.149161,17.113223


# World Weather Online API

World Weather Online API is [here](https://www.worldweatheronline.com/developer/api/historical-weather-api.aspx)

1. Sign up for FREE api key if you haven't done that before (it's free for **30 days**).
2. For each crush, get the weather for the location and date.
3. Find a relationship between the weather and severity of the crash.

Hints:

* pull weather only for smaller sample of crashes (250 or so) due to API limits
* for sending HTTP requests check out "requests" library [here](http://docs.python-requests.org/en/master/)
